<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**pip**
---



In [69]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler
# install parquet libraries
!pip install git+https://github.com/dask/fastparquet
!pip install python-snappy
!pip install pyarrow 

CPU times: user 1.69 s, sys: 292 ms, total: 1.98 s
Wall time: 49 s


**import**
---



In [42]:
import glob
import os
from pathlib import Path
from sys import getsizeof
import dask
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
from dask.distributed import Client
from shapely.geometry import box

In [4]:
%%capture
client = Client(processes=False)
client

In [13]:
%cd /content/drive/My Drive/Colab Notebooks/01_Repository/FFE

/content/drive/My Drive/Colab Notebooks/01_Repository/FFE


In [56]:
pathShapefile = './data/shapefile'
path_output = Path('/content/drive/My Drive/Colab Notebooks/02_Output/ffeFinnComparison')

In [57]:
!ls '../../02_Output/ffeFinnComparison'

scenario1_pid1648_Activations.parquet  scenario1_pid1727_Activations.ftr
scenario1_pid1649_Activations.parquet  scenario1_pid1728_Activations.ftr
scenario1_pid1650_Activations.parquet  scenario1_pid1729_Activations.ftr
scenario1_pid1651_Activations.parquet  scenario1_pid1730_Activations.ftr


**definitions**
---

In [66]:
@dask.delayed
def read_and_concatenate_parquets(path=path_output):
  L = []
  files = path_output.glob('*.parquet')
  for file in files:
    print("file loaded : {}".format(file))
    pqt = dd.read_parquet(file, engine='auto')
    L.append(pqt)
  df = dd.concat(L)
  return df

def count_fid_occurences(df):
  count = df['source'].value_counts().compute()
  count_df = pd.DataFrame({'source': count.index, 'count': count.values})
  return count_df

def load_shapefile(file_name):#, minx, miny, maxx, maxy):
    # crop data
    # bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(pathShapefile,file_name))#, bbox=bbox)
    max_extent = gdf_buildings.total_bounds
    data_size = getsizeof(gdf_buildings) /(1024.0**3)
    print("Shapefile extent : {}".format(max_extent))
    print("Asset loaded : {}".format(len(gdf_buildings)))
    # gdf_buildings.plot(column='IgnProb_bl', cmap='hsv', legend=True)
    return gdf_buildings

def merge_coordinates_export_shape(ddf, gdf, name_output):
  gdf = gdf[['FID', 'geometry']]
  df = pd.DataFrame(gdf)
  # ddf = ddf.compute()
  df_merge = ddf.merge(df, how='left', left_on='source', right_on='FID')
  gdf_merge = gpd.GeoDataFrame(df_merge, geometry='geometry')
  gdf_merge.to_file(os.path.join(path_output, name_output + ".shp"))

  f, ax = plt.subplots(1, figsize=(12, 12))
  countPlot = gdf_merge.plot(column='count', cmap='seismic', legend=True, ax=ax)
  plt.savefig(os.path.join(path_output, 'ComparisonFinn_3000runs.jpg'), dpi=300)
  plt.show()
  return gdf_merge

**runs**
---



In [84]:
df = read_and_concatenate_parquets()

In [81]:
# df.to_parquet(os.path.join(path_output, 'FFE_comparisonFinn_3000runs.parquet'), engine='auto')

Delayed('to_parquet-8063c97d-4c2d-441c-be54-cd91cb1c8714')

In [ ]:
count_df = count_fid_occurences(df)

In [ ]:
count_df.describe()

In [ ]:
gdf = load_shapefile("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small
gdf_count = merge_coordinates_export_shape(count_df, gdf, "smallBurnedBuildings")